# Quantum Error Decoding using NEAT

<div style="text-align: justify">This is my account of the QOSF Mentorship project that I realized in the second quarter of 2021 under supervision of Dr. Dario Rosa. The goal was to implement the following paper titled "A NEAT Quantum Decoder" by Hugo Théveniaut and Evert van Nieuwenburg [1]: https://arxiv.org/abs/2101.08093 and learn about toric code and neuro-evolution along the way. The paper itself presents a new method to conduct active protection of the toric code against random bit- and phase-flips, based on Neuro-Evolutionary Augmented Topologies algorithm. It performs similarly as good as already known methods, but uses a few orders of magnitude less parameters and thus has a significant advantage over them. This notebook is intended to be a gentle introduction to both toric code and NEAT as well as contain my implementation of the paper. However, the last part is still work in progress as I didn't manage to finish the implementation on time and it is going to be added in the future, so for now the aim of the project was fulfilled only partially.</div>

### Part I. Toric Code

In the era of so-called NISQ devices (Noisy Intermediate-Scale Quantum) researchers have to deal with the noise constantly introducing logical errors to qubits. In classical codes where we operate with zeroes and ones only one type of error can occur, namely a bit-flip which is unintentional swap of $0$ into $1$ or the other way around. But in the quantum realm we can in fact distinguish two types of errors.

First one is a <i>bit-flip</i> which transforms an arbitrary quantum state $\alpha |0> + \beta |1>$ to $\beta |0> + \alpha |1>$, so basically it's equivalent to applying $X$ gate to a qubit. And the second one is a <i>phase-flip</i> which transforms an arbitrary quantum state $\alpha |0> + \beta |1>$ to $\alpha |0> - \beta |1>$, so basically it's equivalent to applying $Z$ gate to a qubit, sometimes it's called <i>depolarization</i>.

To prevent that it is possible to encode an information stored by one qubit in many qubits - most often the more the better. There are for example three-qubit encodings of one logical qubit allowing to prevent a single bit-flip and analogously similar encoding for preventing phase-flips. There is also a nine-qubit encoding of one logical qubit called Shor code covering all possible errors, namely bit-flips, phase-flips and their linear combinations as well.

 However, not so long after Shor in 1997 A. Kitaev proposed a new way of such encoding in [2]. Two logical qubits are realized as $2N^2$ physical qubits arranged in a $N \times N$ lattice whose opposite edges are identified (i.e. glued together), hence it creates a torus on which qubits lie. On the picture from [1] I showed which qubits exactly are identified using colored hexagons. On this picture each corner of the chessboard is a physical qubit whose state of course can be changed by some random Pauli operator. Moreover, the toric code is defined as a stabilizer code, i.e. there are operators called stabilizers which don't change the logical state of the lattice by definition. These are plaquette and star which can be applied to squares of the <i>chessboard</i>. They are respectively a product of $Z$ operators on corners of a black square and a product of $X$ operators on corners of a white square. Again, by definition as long as we can return to the empty lattice (without syndromes) with these operators solely, its logical state hasn't change.

<img src="images/img11.png" width=600>

Since they don't change the logical value of the system they can be measured on and on and give us constantly some information on what's going on. The measurement circuit for both types is shown below. And a visualization of a toric code, when the opposite edges are glued together. The following two pictures come from [4].

<img src="images/img13.png" width=600>

<img src="images/img20.png" width=600>

As it follows from simple calculations, plaquette is measured $1$ iff there is an even amount of $X$ operators applied to the square we measure and $-1$ otherwise and vice versa - star is measured $1$ iff there is an even amount of $Z$ operators applied to the square we measure and $-1$ otherwise. What is worth repeating here, it is not possible to measure plaquette or star on each square of the <i>chessboard</i> without disturbing the logical state of the lattice. We can only measure plaquettes on black rows and stars on white rows. If stabilizer for a given square is measured $-1$ we say there's a syndrome and they are depicted on the first picture as orange circles.

So, the only knowledge we can get about the system while measuring those stars and plaquettes is where the syndromes currently are. It is not the full knowledge about the system but it will be sufficient to monitor the possible logical errors and correct its state by applying $X$, $Y$, $Z$ to physical qubits before a logical error occurs. Let's observe that one physical error, i.e. a single qubit flipped, generates two syndromes, but if two neighbouring qubits get the same error, then there are again two syndromes and not four for example. We can also add in a third error and so on and there will be still just two syndromes on the ends of what we call an "error string". Error strings are important because if they get closed creating a loop as these two endpoints meet each other, all syndromes disappear. That's because on each square's corners there is an even number of errors. If the loop goes even number of times through each of vertical and horizontal boundaries, it's called "trivial" and everything is OK, because we can eliminate this loop with stabilizer operators. However, if such a loop goes through boundary as you can see on the right half of the first picture, it is called "non-trivial" and can't be eliminated with plaquettes and stars solely. Thus, it introduces a logical error. Not going into details, the key topological concept here is contraction and it's possible to contract a loop (informally squeeze it to a point) only if it doesn't go around one of torus equators. On the other hand it goes through them exactly when it passes through the glued boundaries as you can check imagining the way a torus is assembled out of a square. Lastly, if there's a vertical non-trivial loop made of bit-flips (phase-flips), the logical state of the first logical qubit is bit-flipped (phase-flipped) and if there's a horizontal non-trivial loop, the state of the second qubit is flipped.

<img src="images/img30.png" width=600>

Now for the coding part. Let's import the necessary libraries.

In [12]:
import numpy as np
import math as m
from random import random, choice
from numpy.random import normal

First we need to define a lattice of physical qubits, which is going to be just a $2N \times N$ matrix consisting of relevant operators applied to different qubits at any moment. These operators can be $X$, $Z$ or $Y$ which is a combination of two former ones. They will be introduced both at random what is intended to simulate the noisy enviroment and by our choice what means applying one of the three aforementioned gates to any of these physical qubits.

In [2]:
def lattice(N):
    # N is the code distance and the lattice is 2N x N
    array = []
    for i in range(2*N):
        array.append([])
    for i in range(2*N**2):
        array[i % (2*N)].append("╳")
    return array

In [3]:
def error(lat, i, j, e):
    # i, j are respectively row and column index of a qubit in the lattice lat
    # e can be one of "X", "Y", "Z" meaning three Pauli operators
    N = len(lat[0])
    if (lat[i%(2*N)][j%N] == e): lat[i%(2*N)][j%N] = "╳"
    elif (lat[i%(2*N)][j%N] == "Y"): 
        if (e == "X"): lat[i%(2*N)][j%N] = "Z"
        else: lat[i%(2*N)][j%N] = "X" 
    elif (lat[i%(2*N)][j%N] != "╳"): lat[i%(2*N)][j%N] = "Y" 
    else: lat[i%(2*N)][j%N] = e

Another fundamental thing we need to code is the syndrome measurement which will be stored in another matrix. The important thing here is that plaquettes and stars can be measured in different rows of our lattice, so in even rows of the resulting matrix we will keep plaquette measurements and in the odd rows we will keep star measurements. If the measurement is $1$ (there's no syndrome) we write $0$ in our matrix and if it's $-1$ (there's a syndrome) we write $1$, the only purpose of such a conversion is our later convenience.

In [7]:
def X(lat):
    # an auxilliary matrix storing information about bit flips on the lattice
    # it keeps -1 in (i, j) iff (i, j) qubit was bit-flipped, otherwise 1
    N = len(lat[0])
    array = lattice(N)
    for i in range(2*N):
        for j in range(N):
            if (lat[i][j] == "X" or lat[i][j] == "Y"): array[i][j] = -1
            else: array[i][j] = 1
    return array

def Z(lat):
    # an auxilliary matrix storing information about phase flips on the lattice
    # it keeps -1 in (i, j) iff (i, j) qubit was phase-flipped, otherwise 1
    N = len(lat[0])
    array = lattice(N)
    for i in range(2*N):
        for j in range(N):
            if (lat[i][j] == "Z" or lat[i][j] == "Y"): array[i][j] = -1
            else: array[i][j] = 1
    return array

def syndromes(lat):
    N = len(lat[0])
    X_lat = X(lat)
    Z_lat = Z(lat)
    array = lattice(N)
    for i in range(2*N):
        for j in range(N):
            if (i%2 == 0):
                array[i][j] = (1-X_lat[i][j]*X_lat[(i+1)%(2*N)][j]*X_lat[(i+2)%(2*N)][j]*X_lat[(i+1)%(2*N)][(j+1)%N])//2
            else:
                array[i][j] = (1-Z_lat[i][j]*Z_lat[(i+1)%(2*N)][j]*Z_lat[(i+2)%(2*N)][j]*Z_lat[(i+1)%(2*N)][(j-1)%N])//2
    return array

Below is a function that will help us visualise the current state of the lattice.

In [8]:
def draw(lat):
    N = len(lat[0])
    syn = syndromes(lat)
    for i in range(2*N):
        line = ""
        if (i%2 == 0):
            if (syn[(i-1)%(2*N)][0] == 1):
                line += "o "
            else:
                line += "  "
            for j in range(N):
                line += lat[i][j]
                if (syn[(i-1)%(2*N)][(j+1)%N] == 1):
                    line += " o "
                else:
                    line += "   "
            print(line)
            line = ""
            for j in range(N):
                line += " / \\"
            print(line)
            line = ""
        else:
            for j in range(N):
                line += lat[i][j]
                if (syn[(i-1)%(2*N)][j%N] == 1):
                    line += " o "
                else:
                    line += "   "
            line += lat[i][0]
            print(line)
            line = ""
            for j in range(N):
                line += " \\ /"
            print(line)
            line = ""
    if (syn[2*N-1][0] == 1):
        line += "o "
    else:
        line += "  "
    for j in range(N):
        line += lat[0][j]
        if (syn[2*N-1][(j+1)%N] == 1):
            line += " o "
        else:
            line += "   "
    print(line)

For example let's generate a $5$x$10$ lattice and put some errors in there.

In [9]:
qubits = lattice(5)
error(qubits, 0, 3, "X")
error(qubits, 1, 1, "Z")
error(qubits, 2, 4, "Y")
draw(qubits)

  ╳ o ╳   ╳   X   ╳   
 / \ / \ / \ / \ / \
╳   Z   ╳   ╳ o ╳ o ╳
 \ / \ / \ / \ / \ /
o ╳ o ╳   ╳   ╳ o Y o 
 / \ / \ / \ / \ / \
╳   ╳   ╳   ╳   ╳ o ╳
 \ / \ / \ / \ / \ /
  ╳   ╳   ╳   ╳   ╳   
 / \ / \ / \ / \ / \
╳   ╳   ╳   ╳   ╳   ╳
 \ / \ / \ / \ / \ /
  ╳   ╳   ╳   ╳   ╳   
 / \ / \ / \ / \ / \
╳   ╳   ╳   ╳   ╳   ╳
 \ / \ / \ / \ / \ /
  ╳   ╳   ╳   ╳   ╳   
 / \ / \ / \ / \ / \
╳   ╳   ╳   ╳ o ╳   ╳
 \ / \ / \ / \ / \ /
  ╳ o ╳   ╳   X   ╳   


As you see syndromes appear inside the squares which have an odd number of qubits bit-flipped or phase-flipped and the first type of syndromes is found in even rows only while the other one in odd rows only. The next thing we want to implement is random bit-flip and depolarizing noise, which correspond to random $X$ and $Z$ operations on physical qubits.

In [10]:
def randombitflips(lat, prob):
    # prob is the probability for each physical qubit to be flipped
    N = len(lat[0])
    for i in range(2*N):
        for j in range(N):
            r = np.random.rand()
            if (r < prob): error(lat, i, j, "X")

def randomdepolarizing(lat, prob):
    # prob is the probability for each physical qubit to be depolarized
    N = len(lat[0])
    for i in range(2*N):
        for j in range(N):
            r = np.random.rand()
            if (r < prob/3): error(lat, i, j, "X")
            elif (r < 2*prob/3): error(lat, i, j, "Z")
            elif (r < prob): error(lat, i, j, "Y")

Now let's check how does it work:

In [12]:
qubits10 = lattice(10) 
randombitflips(qubits10, 0.5)
draw(qubits10)

  X   ╳   X   X   X   X   X   X   X   ╳   
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
╳   X o ╳ o ╳   ╳   ╳ o ╳ o X   X   ╳ o ╳
 \ / \ / \ / \ / \ / \ / \ / \ / \ / \ /
  ╳   ╳   ╳   X   X   ╳   X   X   ╳   X   
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
╳   X o ╳   X o ╳   X   X   ╳ o X o ╳   ╳
 \ / \ / \ / \ / \ / \ / \ / \ / \ / \ /
  X   ╳   X   X   ╳   ╳   ╳   X   ╳   X   
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
X o ╳ o ╳ o ╳   ╳ o ╳ o X   X   X o ╳   X
 \ / \ / \ / \ / \ / \ / \ / \ / \ / \ /
  X   X   ╳   X   X   ╳   ╳   X   ╳   ╳   
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
╳   X   ╳   X   ╳   ╳   ╳ o ╳   X o ╳ o ╳
 \ / \ / \ / \ / \ / \ / \ / \ / \ / \ /
  ╳   ╳   X   ╳   X   ╳   X   ╳   ╳   X   
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
X o ╳ o ╳   ╳   X o X   ╳ o ╳ o ╳   ╳ o X
 \ / \ / \ / \ / \ / \ / \ / \ / \ / \ /
  ╳   X   X   X   ╳   X   ╳   X   ╳   X   
 / \ / \ / \ / \ / \ / \ / \ / \ / \ / \
╳ o ╳ o ╳   ╳ o X   ╳ o ╳   ╳ o ╳ o ╳   ╳
 \ / \ / \ / \ / \ / \ / \ / \ / \ / \ 

In [13]:
qubits7 = lattice(7)
randomdepolarizing(qubits7, 0.25)
draw(qubits7)

  Z   ╳   ╳   ╳   ╳   X o Y   
 / \ / \ / \ / \ / \ / \ / \
╳   Z   ╳   ╳   ╳ o ╳ o ╳ o ╳
 \ / \ / \ / \ / \ / \ / \ /
o ╳ o ╳   ╳   ╳   Y   ╳ o Z o 
 / \ / \ / \ / \ / \ / \ / \
╳   ╳   ╳ o ╳   Z o Z o ╳ o ╳
 \ / \ / \ / \ / \ / \ / \ /
o ╳ o Z   Y o ╳   ╳ o X o Y o 
 / \ / \ / \ / \ / \ / \ / \
╳   ╳ o X   ╳ o Y o ╳ o ╳ o ╳
 \ / \ / \ / \ / \ / \ / \ /
  ╳   ╳   ╳   ╳ o ╳   ╳   ╳   
 / \ / \ / \ / \ / \ / \ / \
╳   ╳ o ╳   ╳ o ╳   ╳   ╳   ╳
 \ / \ / \ / \ / \ / \ / \ /
  ╳   X   ╳   X   ╳   ╳ o ╳   
 / \ / \ / \ / \ / \ / \ / \
╳   ╳ o ╳ o ╳ o ╳   ╳ o Z o ╳
 \ / \ / \ / \ / \ / \ / \ /
o ╳ o Z o X o Z   ╳ o Y o Y o 
 / \ / \ / \ / \ / \ / \ / \
╳   ╳   ╳ o ╳   Z   ╳ o ╳ o ╳
 \ / \ / \ / \ / \ / \ / \ /
  ╳   ╳ o Z o ╳ o ╳   ╳   ╳   
 / \ / \ / \ / \ / \ / \ / \
╳   ╳ o X o ╳   ╳   ╳ o ╳ o ╳
 \ / \ / \ / \ / \ / \ / \ /
  Z   ╳   ╳   ╳   ╳   X o Y   


Couple more functions below.

In [14]:
def hamiltonian(lat):
    # computes the hamiltonian of the system
    N = len(lat[0])
    syn = syndromes(lat)
    amount = 0
    for i in range(2*N):
        for j in range(N):
            if (syn[i][j] != 1): amount -= 1
    return amount

def plaquette(lat, i, j):
    # (i,j) points to the square in i-th row and j-th column
    N = len(lat[0])
    if (i%2 == 0):
        error(lat, i, j, "Z")
        error(lat, (i+1)%(2*N), j, "Z")
        error(lat, (i+2)%(2*N), j, "Z")
        error(lat, (i+1)%(2*N), (j+1)%N, "Z")
    else: print("You cannot place a plaquette there.")
    
def star(lat, i, j):
    # (i,j) points to the square in i-th row and j-th column 
    N = len(lat[0])
    if (i%2 == 1):
        error(lat, i, j, "X")
        error(lat, (i+1)%(2*N), j, "X")
        error(lat, (i+2)%(2*N), j, "X")
        error(lat, (i+1)%(2*N), (j-1)%N, "X")
    else: print("You cannot place a star there.")

The hamiltonian of such a lattice is here just energy of its state. Namely, an empty lattice has the energy of $-2N^2$ and gets bigger by $1$ with each new syndrome, so that a lattice with syndromes inside all its squares has the energy of $0$. We're not gonna use it, but it's a nice physical intepretation what quantum decoding really means - it's minimizing the energy of a given quantum state without introducing any logical errors. We can for example check the hamiltonian of the last drawn lattice and an empty equivalent.

In [17]:
print(hamiltonian(qubits7))
print(hamiltonian(lattice(7))) # should be -2*7^2

-52
-98


### Part II. NeuroEvolution of Augmenting Topologies

Standard methods to adjust a neural network to perform a specific task are done with some sort of gradient involved. However, in the case of evolutionary algorithms we don't need it. For a task to be trained on we need a game which a neural network can play and get a positive reward if it succeeds (win) or a negative one otherwise (lose). In our case it's going to be a toric code decoding game, described in Part III. Changes to neural networks in some considered population are done by random mutations and they are directed by positive or negative feedback from the game all neural networks play. These which win the most often are passed to the next generation and then combined with others winners and replicated. It is only possible due to the notion of a genome of a neural network. Moreover, NEAT allows to evolve and adjust not only weights and biases but also the topology of a network, i.e. its shape, nodes, connections. The original paper were NEAT algorithm was first introduced is [3]. All images from now on come from [1].

NEAT step-by-step goes like this: 
1. Initialize a population of trivial neural networks.
2. Measure what percentage of games do these neural networks win.
3. Mutate them randomly with a certain probability.
4. Based on 2. leave the best individuals only.
5. Divide them into species, which means briefly group them by similarity.
6. Crossover individuals within species to get a new population.
7. Repeat steps 2. to 6. how many times you want.
8. Choose the best individual basing on its percentage of won games.

The crucial feature of NEAT is the genomic encoding of a neural network. In principle any neural network can be described in terms of a genome in which each gene corresponds to a node or connection and its properties.

<img src="images/img3.png" width=500>

Such an encoding allows us to define easily various types of mutations what I'll touch later. For example a node mutation can occur in which a random edge is divided on two edges and a new hidden neuron between them is created.

<img src="images/img4.png" width=400>

The heart of the algorithm is crossover which given two genomes produces its offspring with parental features inherited randomly.

<img src="images/img1.png" width=500>

In order to implement NEAT algorithm we need to agree on what does a genome look like and how are we going to define it as part of the code. Well, let's say it's a 2-element list whose first element is a list of all node genes and analogously the second one is a list of all connection genes. Furthermore, a node gene is a list of the node properties as follows: $[\text{id}, \text{type}, \text{bias}]$, where $\text{id}$ is identification number, $\text{type}$ is "in", "out" or "hid" which correspond to different types of nodes in a neural network and $\text{bias}$ is just... well, a bias of that node, i.e. a number that is added to the weighted sum of signals from preceding neurons, but before the neuron activation is computed. Input nodes have always bias equal to 0. Whereas a connection gene of a connection between nodes $\text{A}$ and $\text{B}$ is a list of the connection properties as follows: $[\text{innovation number}, \text{id of node A}, \text{id of node B}, \text{weight}, \text{status}]$. Innovation number is a unique number assigned to a connection gene whenever a new connection is created via mutation. This way we can easily track the history of the evolution history. Status can be $0$ or $1$ and means just "turned off" or "turned on" respectively.

To give an example, a very simple neural network consisting of two input neurons which are connected to a hid neuron, which is then connected to an output neuron, might have a following genome: 

<center>$[[[0, \text{"in"}, 0],[1, \text{"in"}, 0],[2, \text{"hid"}, 3.5],[3, \text{"out"}, 0.2]],[[0, 0, 2, 0.1, 1],[1, 1, 2, 0.4, 1],[2, 2, 3, 0.7, 1]]]$</center>

For understanding the further code it's important to remember that $\text{genome}[0]$ refers here to the list of nodes genes and $\text{genome}[1]$ refers here to the list of connection genes. Below is defined a function that for given genome and inputs it returns the probabilities for each of output neurons for a neural network described by the genome.

In [3]:
def sigmoid(x):
    # it will be used as an activation function in hidden and output neurons
    return 1 / (1 + m.exp(-x))

def compute(genome, nodes, k):
    # a subroutine to compute a value of a k^th neuron if all its inputs are known
    weighted_sum = 0
    for y in genome[1]:
        if (y[4] == 1 and y[2] == k):
            if (nodes[y[1]] == "empty"):
                return "empty"
            weighted_sum += y[3] * nodes[y[1]]
    result = sigmoid(weighted_sum + genome[0][k][2])
    return result

def evaluate(genome, inputs):
    # it does the forward propagation given genome and inputs and returns outputs
    nodes = []
    
    for x in genome[0]:
            nodes.append("empty")
    
    i = 0
    for k in range(len(nodes)):
        if (genome[0][k][1] == "in"):
            nodes[k] = inputs[i]
            i += 1
    if (i != len(inputs)):
        print("Number of input neurons is wrong.")
    
    while ("empty" in nodes):
        for k in range(len(nodes)):
            if (nodes[k] == "empty"):
                nodes[k] = compute(genome, nodes, k)
    
    outputs = []
    for i in range(len(genome[0])):
        if (genome[0][i][1] == "out"):
            outputs.append(nodes[i])
    return outputs

For later implementation we'll need a so-called trivial network that takes $9$ inputs and outputs probabilities of $4$ possible choices in a random way, i.e. biases and weights are generated by the normal distribution. The initial population will consist of trivial networks only.

In [4]:
def trivial_network():
    # the initial values of weights and node biases are taken from gaussian distribution N(0,1)
    first = [[0, "in"], [1, "in"], [2, "in"], [3, "in"], [4, "in"], [5, "in"], [6, "in"], [7, "in"], [8, "in"], [9, "out"], [10,"out"], [11, "out"], [12, "out"]]
    second = []
    for i in range(9):
        first[i].append(0)
        for j in range(4):
            weight = normal()
            second.append([len(second), i, 9+j, weight, 1])
    for i in range(4):
        bias = normal()
        first[9+i].append(bias)
    genome = [first, second]
    return genome

Next thing is a mutation procedure that takes a genome and modifies it in one of $8$ possible ways: either adds a new connection or removes an existing connection or adds a node or removes a node or changes a weight or changes a bias or enables a connection or disables a connection.

In [ ]:
def mutate(genome, max_node, max_conn):
    # max_node is the largest node index present in the whole population
    # max_conn is the largest innovation number in the whole population
    r = random()
    if (r < 0.1):
        #add connection
        pairs = []
        for x1 in genome[0]:
            for x2 in genome[0]:
                if (("in" in x1 and "in" in x2) or ("out" in x1 and "out" in x2)):
                    continue
                areConnected = 0
                for y in genome[1]:
                    if (y[1] == x1[0] and y[2] == x2[0]):
                        areConnected = 1
                if (areConnected == 0):
                    pairs.append([x1[0], x2[0]])
        if (len(pairs) != 0):
            max_conn += 1
            p = choice(pairs)
            r = normal()
            genome[1].append([max_conn, p[0], p[1], r, 1])
        
    elif (r < 0.2):
        #remove connection
        y = choice(genome[1])
        genome[1].remove(y)
        
    elif (r < 0.3):
        #add node
        max_node += 1
        max_conn += 2
        y = choice(genome[1])
        r = normal()
        genome[0].append([max_node, "hid", r])
        r = normal()
        genome[1].append([max_conn-1, y[1], max_node, r, 1])
        r = normal()
        genome[1].append([max_conn, max_node, y[2], r, 1])
        y[4] = 0
        
    elif (r < 0.4):
        #remove node
        hidden = []
        for x in genome[0]:
            if ("hid" in x):
                hidden.append(x)
        if (len(hidden) != 0):
            x = choice(hidden)
            connections = []
            for y in genome[1]:
                if (y[1] == x[0] or y[2] == x[0]):
                    connections.append(y)
            for y in connections:
                genome[1].remove(y)
            genome[0].remove(x)
        
    elif (r < 0.9):
        #weight mutation
        y = choice(genome[1])
        r = normal()
        y[3] = r
        
    else:
        #bias mutation
        x = [0, "in", 0]
        while (x[1] == "in"):
            x = choice(genome[0])
        r = normal()
        x[2] = r
        
    r = random()
    if (r < 0.01):
        #enable connection
        disabled = []
        for y in genome[1]:
            if (y[4] == 0):
                disabled.append(y)
        if (len(disabled) != 0):
            y = choice(disabled)
            y[4] = 1
    elif (r < 0.02):
        #disable connection
        enabled = []
        for y in genome[1]:
            if (y[4] == 1):
                enabled.append(y)
        if (len(enabled) != 0):
            y = choice(enabled)
            y[4] = 0
            
    result = [genome, max_node, max_conn]
    
    return result

And the last on our list in this part is crossover. It's meant to imitate a biological crossover where, in order to create a new unique organism, genes of parents are combined together. Thus, if in both parental genomes there is a specific innovation number, a new genome inherits one of these genes with that number (other parameters might be still different) chosen randomly. The other connection genes, which are present only in one of parental genomes, are inherited randomly.

In [ ]:
def crossover(genome1, genome2):
    first = []
    second = []
    
    for x in genome1[0]:
        isyFound = 0
        for y in genome2[0]:
            if (x[0] == y[0]):
                r = random()
                if (r < 0.5):
                    first.append(x)
                else:
                    first.append(y)
                isyFound = 1
        if (isyFound == 0):
            first.append(x)
        
    for x in genome1[1]:
        isyFound = 0
        for y in genome2[1]:
            if (x[0] == y[0]):
                r = random()
                if (r < 0.5):
                    second.append(x)
                else:
                    second.append(y)
                isyFound = 1
        if (isyFound == 0):
            r = random()
            if (r < 0.5):
                second.append(x)
                
    for x in first:
        isConnected = 0
        for y in second:
            if (y[1] == x[0] or y[2] == x[0]):
                isConnected = 1
                break
        if (isConnected == 0):
            first.remove(x)
                
    genome = [first, second]
    return genome

Moreover, in order to protect new innovations that occured in the population, all neural networks are divided into species. A distance function $\delta = c_1 \frac{E}{N_\text{genes}} + c_2 \frac{D}{N_\text{genes}} + c_3 + \bar{W}$ between two neural networks is introduced and basically all representants which are distant from a given neural network for less than some agreed upon threshold $\delta_c$ belong to the same species. In the formula $E$ stands for excess genes between two genomes and similarly $D$ for disjoint genes, whereas $\bar{W}$ is the average weight difference in the shared genes. And $N_\text{genes}$ means the number of genes in the largest genome. The aim is to allow newly-mutated properties to adjust in a less-agressive environment before they are released to the whole population. For instance if there's a new connection it allows it to get better weight to be actually useful and contributing to the whole evolution process, instead of being dismissed before its potential would be unleashed.

### Part III. Implementation

Now, when we already know the theory behind toric code and NEAT algorithm, we can eventually apply the latter to the former to obtain fault-tolerant encoding of qubits. In order to do so we need first to implement a subroutine checking is there any non-trivial loop in the lattice we work with. Next we can define the toric code decoding game, which can be summed up in two steps: 1. Apply a Pauli gate to one of qubits. 2. Check is there any non-trivial loop. If so, the game is lost. If not, go to the step 1. Repeat these steps until all syndromes disappear.

<img src="images/img10.png" width=750>

Above you can see how Correction #1 does not introduce a non-trivial loop, while Correction #2 does. Below is a subroutine defined that starts at each of upper squares of the lattice and explores it in the form of a ternary tree along all error strings which go through those squares.

In [2]:
def extend(input_list, index, value):
    if (index > len(input_list)-1):
        for i in range(index-len(input_list)+1):
            input_list.append(0)
    input_list[index] = value

def newleaf(input_list, parent, value):
    if(3*parent+4 > len(input_list)):
        extend(input_list, 3*parent+3, 0)
    for i in range(3):
        if (input_list[3*parent+i+1] == value):
            break
        if (input_list[3*parent+i+1] == 0):
            input_list[3*parent+i+1] = value
            break
        if (i == 2): print("This parent already possesses 3 leaves.")

def Xiteration(lat, tab, n):
    # n is index of a vertex in the tree 'tab'
    if (n > len(tab)-1 or tab[n] in [0, "loop", "comeback"]): return None
    N = len(lat[0])
    i = tab[n][0]               #coords of a given vertex
    j = tab[n][1]
    g = tab[m.floor((n-1)/3)][0]    #coords of a given vertex' parent
    h = tab[m.floor((n-1)/3)][1]
    if ((i != g or (j-1)%N != h) and (lat[(i+1)%(2*N)][j] in ["X", "Y"])):
        newleaf(tab, n, [i, (j-1)%N])
    if ((i != g or (j+1)%N != h) and (lat[(i+1)%(2*N)][(j+1)%N] in ["X", "Y"])):
        newleaf(tab, n, [i, (j+1)%N])
    if (((i+2)%(2*N) != g or j != h) and (lat[(i+2)%(2*N)][j] in ["X", "Y"])):
        newleaf(tab, n, [(i+2)%(2*N), j%N])
    if (((i-2)%(2*N) != g or j != h) and (lat[i][j] in ["X", "Y"])):
        newleaf(tab, n, [(i-2)%(2*N), j])

Another function which if a square is met twice along one of branches of the tree walk tells us is it a trivial or non-trivial loop.

In [6]:
def isLooped(lat, tab, n):
    # n is index of a vertex in the tree 'tab'
    N = len(lat[0])
    i = n
    counter = 0     #number of times the horizontal boundary was crossed
    while (i > 0):
        if (tab[i][0] - tab[m.floor((i-1)/3)][0] == 2*(N-1)):
            counter -= 1
        if (tab[i][0] - tab[m.floor((i-1)/3)][0] == 2*(-N+1)):
            counter += 1
        if (i > 0):
            i = m.floor((i-1)/3)
        if (tab[i] == tab[n]):
            if (i == 0 and counter%2 == 1):
                return "comeback"
            else:
                return "loop"
    return tab[n]

And at last a function that returns $0$ iff there is a non-trivial loop going from up to down and after considering also the rotation of the lattice we get the function for both types of non-trivial loops - vertical ones as well as horizontal ones. 

In [7]:
def vertical_reward(lat):
    N = len(lat[0])
    k = 1
    walk = []
    maxwalk = 2
    
    for j in range(N):
        walk.append([])
        walk[j].append([0, j])
        if(lat[1][j] in ["X","Y"]):
            walk[j].append([0,(j-1)%N])
        if(lat[1][(j+1)%N] in ["X","Y"]):
            walk[j].append([0,(j+1)%N])
        if(lat[2][j] in ["X","Y"]):
            walk[j].append([2,j])
            
    while (k < maxwalk and k < 3**(N**2+1)):
        for i in range(N):
            Xiteration(lat, walk[i], k)
            if (3*k+3 < len(walk[i])):
                for l in [3*k+1, 3*k+2, 3*k+3]:
                    if (walk[i][l] != 0):
                        walk[i][l] = isLooped(lat, walk[i], l)
                        #print(k, i, walk[i][l])
                        if (walk[i][l] == "comeback"): 
                            return 0
            if (len(walk[i]) > maxwalk):
                maxwalk = len(walk[i])
        k += 1
    #print(walk)
    return 1

def transpose(lat):
    N = len(lat[0])
    tab = []
    for i in range(2*N):
        tab.append([])
    for j in range(N):
        for i in range(N):
            tab[2*j].append(lat[2*N-1-2*i][j])
            tab[2*j+1].append(lat[2*N-1-2*i-1][j])
    return tab

def reward(lat):
    return vertical_reward(lat) * vertical_reward(transpose(lat))

This reward can now be used for implementing the mentioned game as all choices that our networks will undertake can be quantatively measured with rewarding it for good choices.

<i> THE BELOW IMPLEMENTATION OF NEAT IS WORK IN PROGRESS </i>

In [ ]:
def empty(syn):
    for row in syn:
        if (1 in row):
            return 0
    return 1

def vertex(i):
    if (i == 0): return [0, 0]
    if (i == 1): return [1, 0]
    if (i == 2): return [1, 1]
    if (i == 3): return [2, 0]
    
def best(probabilities):
    # [probabilities for indices 0..4]
    highest = [0, 0]
    for i in range(len(probabilities)):
        if (probabilities[i] > highest[1]):
            highest = [i, probabilities[i]]
    return highest 

def syndrome_neighborhood(syn, i, j):
    N = len(syn[0])
    synhood = []
    synhood.append(syn[(i-2)%(2*N)][(j-1)%N])       #0
    synhood.append(syn[(i-2)%(2*N)][j])             #1
    synhood.append(syn[(i-2)%(2*N)][(j+1)%N])       #2
    synhood.append(syn[i][(j-1)%N])                 #3
    synhood.append(syn[i][j])                       #4
    synhood.append(syn[i][(j+1)%N])                 #5
    synhood.append(syn[(i+2)%(2*N)][(j-1)%N])       #6
    synhood.append(syn[(i+2)%(2*N)][j])             #7
    synhood.append(syn[(i+2)%(2*N)][(j+1)%N])       #8
    return synhood

def game(genome, N, prob, training):
    # for training purpose use training == 1
    # for performance evaluation use training == 0
    
    state = lattice(N)
    randombitflips(state, prob)
    syn = syndromes(state)
    prob_max = 0
    record = [] # for keeping a record of already taken actions
    k = 0
    
    while(not empty(syn)):
        
        for i in range(N):
            for j in range(N):
                synhood = syndrome_neighborhood(syn, 2*i, j)
                outcome = best(evaluate(genome, synhood))
                # [index(0..4), probability]
                if (outcome[1] > prob_max):
                    prob_max = outcome[1]
                    [i_max, j_max] = [2*i + vertex(outcome[0])[0], j + vertex(outcome[0])[1]]
                        
        if (training == 1 and [i_max, j_max] in record):
            return 0
        
        if (training == 0 and k > 1000):
            return 0
        
        error(state, i_max, j_max, "X")
        record.append([i_max, j_max])
        syn = syndromes(state)
        k += 1
        
    return reward(state)

def fitness(genome, N, prob, N_g):
    # N_g is number of games which are played to determine fitness
    points = 0
    for i in range(N_g):
        points += game(genome, N, prob, 0)
        print("Game", i+1, "was played.")
    return points/N_g

def distance(genome1, genome2):
    c_1 = 1.0
    c_2 = 1.0
    c_3 = 0.4
    # to be written
    

def NEAT(pop_size, num_generations, N, error_prob, N_g):
    # hyperparameters from the original paper:
    # pop_size from 100 to 300
    # num_generations = ?
    # number of games N_g = 400
    # error_prob in 0.01, 0.05, 0.1, 0.15 (50 per each)
    population = []
    for i in range(pop_size):
        population.append(trivial_network())
    parents = []
    max_node = len(population[0][0])-1 # the biggest neuron index in the population
    max_conn = len(population[0][1])-1 # the biggest innovation number in the population
    top = 5 # an arbitrary choice of how many top individuals we take for crossover
    threshold_distance = 3.0
    
    for k in range(num_generations):
        for genome in population:
            performance = []
            performance.append(fitness(genome, N, error_prob, N_g))
            mutation = mutate(genome, max_node, max_conn)
            max_node = mutation[1]
            max_conn = mutation[2]
        for j in range(top):
            parents.append(population[best(performance)[0]])
            performance[best(performance)[0]] = 0
        population = []
        for i in range(pop_size):
            a = m.floor(random()*top)
            b = m.floor(random()*top)
            while(a == b):
                b = m.floor(random()*top)
            population.append(crossover(parents[a],parents[b]))
            
    for network in population:
        performance = []
        performance.append(fitness(network, N, error_prob, N_g))
        
    return population[best(performance)[0]]

### References

[1] H. Théveniaut, E. van Nieuwenburg <i>A NEAT Quantum Error Decoder</i>, January 2021, https://arxiv.org/abs/2101.08093

[2] A. Kitaev <i>Fault-tolerant quantum computation by anyons</i>, July 1997, https://arxiv.org/abs/quant-ph/9707021

[3] K. Stanley <i>Evolving Neural Networks through Augmenting Topologies</i>, June 2002, http://nn.cs.utexas.edu/downloads/papers/stanley.ec02.pdf

[4] K. Fujii <i>Quantum Computation with Topological Codes</i>, April 2015, https://arxiv.org/pdf/1504.01444.pdf